# Object Tracking with Tracking APIs and OpenCV

In [1]:
import cv2

## TLD Tracker
- Stands for Tacking, Learning, and Detection.
- The Tracker follows the object frame to frame.
- The Detector localizes all appeareances that have been observed so far & corrects the Tracker if necessary.
- The Learning estimates the Detector's errors and updates it to avoid these errors in the future. 

Pros:
- Good at tracking even with obstruction in the frames (Really useful for a Gym setting).
- Tracks well under large changes in scale and fast moving objects (Useful for sports). 

Cons:
- Can provide many false positives (May track something that is no longer our original object. May be an issue with many people in the setting.)

In [2]:
def chooseTracker():
    print("Tracker APIs Menu")
    print("----------------------")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select a Tracker: ")
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()
        
    return tracker

In [3]:
def My_Tracker():
    """
    Tracks a single object given a tracker
    """
    #tracker = cv2.TrackerTLD_create()
    tracker = chooseTracker()
    tracker_name = str(tracker).split()[0][1:]

    # Open the default camera
    cap = cv2.VideoCapture(0)

    # Check if we succeeded
    if not cap.isOpened():
        raise AssertionError("Camera not found")

    # Capture the first frame
    retval, frame = cap.read() # cv2.VideoCapture.read([image]) → retval, image

    # Select ROI (To manaully draw out the bounding box)
    fromCenter = False
    roi = cv2.selectROI(frame, fromCenter)

    # Initialize tracker with first frame and bounding box
    retval = tracker.init(frame, roi)

    # Keep reading new frames
    while True:
        # Read a new frame
        retval, frame = cap.read()

        # Update Tracker
        success, roi = tracker.update(frame)

        # roi is an array of 4 floating point values
        # To plot out the rectangle, we need each value as integers
        (x,y,w,h) = tuple(map(int,roi))

        # Draw ROI Rectangle as Tracker moves
        if success:
            # If we are able to find and update our Tracker success
            point1 = (x, y) #Top left
            point2 = (x+w, y+h) #Bottom Right
            cv2.rectangle(frame, point1, point2, (0,255,0), 3)
        else:
            # If tracking failed 
            cv2.putText(frame, "Failed to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_DUPLEX, 1,(0,0,255),3)

        # Display tracker type
        cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_DUPLEX, 1, (0,255,0),3)

        # Display result
        cv2.imshow(tracker_name, frame)

        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27: break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
# tracker = cv2.TrackerTLD_create()
# My_Tracker(tracker)
My_Tracker()

Tracker APIs Menu
----------------------
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select a Tracker:  3


error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\highgui\src\window.cpp:356: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [9]:
def Multi_Tracker():
    """
    Tracks multiple objects (Trying to make a Basketball tracker)
    """
    trackers = cv2.MultiTracker_create()
    
#     # Set video to load
#     videoPath = "Videos/LeftAngle.mp4"

#     # Create a video capture object to read videos
#     cap = cv2.VideoCapture(videoPath)

    cap = cv2.VideoCapture(0)
    

    if not cap.isOpened():
        raise AssertionError("Camera not found")

    retval, frame = cap.read() 

    # Select the ROI of each object you want to track (press ENTER or SPACE after selecting the ROI)
    ROIs = cv2.selectROIs("Frame", frame, fromCenter = False)
    ROIs = tuple(map(tuple, ROIs)) 
    # Create a new object tracker for the bounding box and add it to our multi-object tracker
    for roi in ROIs:
        trackers.add(cv2.TrackerTLD_create(), frame, roi)

    #OpenCV uses BGR color space..
    colors = {'green': (0,255,0) , 'blue': (255,0,0), 'red': (0,0,255)}

    # Keep reading new frames
    while True:
        retval, frame = cap.read()
        
        if frame is None:
            break

        # Update Tracker
        (success, ROIs) = trackers.update(frame)
        key = cv2.waitKey(1) & 0xff
        
        if success:
            for roi in ROIs:       
                (x, y, w, h) = tuple(map(int,roi))
                point1 = (x, y) #Top left
                point2 = (x+w, y+h) #Bottom Right
                cv2.rectangle(frame, point1, point2, colors['green'], 2)
        else:
            # If tracking failed 
            cv2.putText(frame, "Failed to Detect all Objects!", (100,200), cv2.FONT_HERSHEY_DUPLEX, 1, colors['red'],3)

        cv2.putText(frame, 'Frame', (20,400), cv2.FONT_HERSHEY_DUPLEX, 1, colors['blue'],3)
        cv2.imshow('Frame', frame)

        #key = cv2.waitKey(1) & 0xff
        if key == 27: break # If ESC pressed

    cap.release()
    cv2.destroyAllWindows()

In [10]:
Multi_Tracker()